## Imports + path settings

In [206]:
from sqlalchemy import create_engine
import os
import pandas
import ipywidgets as widgets
from ipywidgets import interact
import altair
from IPython import display
from config import DB_FOLDER_PATH_S

In [3]:
path_root = os.path.abspath(DB_FOLDER_PATH_S)
path_root

'/home/jovyan/work'

In [4]:
path_db = os.path.join(path_root, "ex3_db", "chinook.db")
path_db

'/home/jovyan/work/ex3_db/chinook.db'

In [5]:
engine = create_engine(f'sqlite:///{path_db}')
# sql = "select count(*) from invoice_items"
# results = engine.execute(sql)
# results.fetchall()

## Pandas

### Tables

In [6]:
sql = 'SELECT name from sqlite_master where type= "table"'
df = pandas.read_sql(sql, con=engine)
df

,name
0,albums
1,sqlite_sequence
2,artists
3,customers
4,employees
5,genres
6,invoices
7,invoice_items
8,media_types
9,playlists


### Invoices

In [7]:
sql = "select * from invoices where CustomerId=2"
df = pandas.read_sql(sql, con=engine)
df.head(100)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,12,2,2009-02-11 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,13.86
2,67,2,2009-10-12 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,8.91
3,196,2,2011-05-19 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
4,219,2,2011-08-21 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,3.96
5,241,2,2011-11-23 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,5.94
6,293,2,2012-07-13 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,0.99


### Customers

In [8]:
sql = "select * from customers LIMIT 5"
df_customers = pandas.read_sql(sql, con=engine)
df_customers.head(100)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


### SELECT Join

In [209]:
def get_invoices_by_client_name(client_name):
    sql1 = "select strftime('%Y-%M-%d',invoices.InvoiceDate) as InvoiceDate,invoices.Total from invoices "
    sql2 = "INNER JOIN customers ON invoices.CustomerId=customers.CustomerId WHERE customers.LastName='"
    sql_client_name = client_name
    sql3 = "' LIMIT 15"
    sql = sql1 + sql2 + sql_client_name + sql3
    return pandas.read_sql(sql, con=engine)

df_invoices_by_client = get_invoices_by_client_name('Gonçalves')
df_invoices_by_client

,InvoiceDate,Total
0,2010-00-11,3.98
1,2010-00-13,3.96
2,2010-00-15,5.94
3,2011-00-06,0.99
4,2012-00-27,1.98
5,2012-00-07,13.86
6,2013-00-07,8.91


### Widgets

In [223]:
def print_chart(customer_name):
    df = get_invoices_by_client_name(customer_name)
    chart = altair.Chart(df).mark_bar().encode(
    x='InvoiceDate',
    y='Total').properties(title = f'{w_client.value}')
    return chart

In [226]:
sql = "select LastName from customers LIMIT 25"
df_customers_names = pandas.read_sql(sql, con=engine)
customers_list = df_customers_names['LastName'].tolist()

w_client = widgets.Dropdown(
    options=customers_list,
    description='Customer:',
    disabled=False,
)

In [227]:
interact(print_chart, customer_name = w_client, );

interactive(children=(Dropdown(description='Customer:', options=('Gonçalves', 'Köhler', 'Tremblay', 'Hansen', …

In [13]:
from platform import python_version

print(python_version())

3.7.6
